In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
% cd /content/drive/MyDrive

/content/drive/MyDrive


In [20]:
data_train = pd.read_csv('./data/train.csv', index_col=0)
data_test = pd.read_csv('./data/test.csv', index_col=0)
data_responses = pd.read_csv('./data/gender_submission.csv', index_col=0)
data_test = pd.concat([data_test, data_responses], axis = 1)
data = pd.concat([data_train, data_test])

In [ ]:
data.isnull().sum()

Survived       0
Pclass         0
Name           0
Sex            0
Age          263
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [21]:
data['Cabin'] = data['Cabin'].fillna('Unknown')
data['Age'] = data['Age'].fillna(data['Age'].mean())
data['Embarked'] = data['Embarked'].fillna('U')
data['Fare'] = data['Fare'].fillna(data['Fare'].mean())

In [22]:
data.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [23]:
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Survived']

data_cleaned = data[columns]

In [24]:
data_cleaned.Pclass=np.where(data_cleaned.Pclass== 1,'First',
                   np.where(data_cleaned.Pclass==2,
                            'Second', 'Third'))
data_cleaned['Cabin_class'] = data_cleaned.Cabin.str[0]
data_cleaned['isfemale']=np.where(data_cleaned.Sex=='female', 1, 0)
data_cleaned.drop(['Sex', 'Cabin'], axis = 1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [42]:
data_cleaned_dummies = pd.get_dummies(data_cleaned,drop_first=True,
                                      columns=['Pclass', 'Cabin_class', 'Embarked'])

In [43]:
X = data_cleaned_dummies.drop(['Survived'], axis= 1)
#'Pclass_First','Cabin_class_A','Embarked_C'
y = data_cleaned_dummies.Survived

In [56]:
X.head(10)

,Age,SibSp,Parch,Fare,isfemale,Pclass_Second,Pclass_Third,Cabin_class_B,Cabin_class_C,Cabin_class_D,Cabin_class_E,Cabin_class_F,Cabin_class_G,Cabin_class_T,Cabin_class_U,Embarked_Q,Embarked_S,Embarked_U
PassengerId,,,,,,,,,,,,,,,,,,
1,22.000000,1,0,7.2500,0,0,1,0,0,0,0,0,0,0,1,0,1,0
2,38.000000,1,0,71.2833,1,0,0,0,1,0,0,0,0,0,0,0,0,0
3,26.000000,0,0,7.9250,1,0,1,0,0,0,0,0,0,0,1,0,1,0
4,35.000000,1,0,53.1000,1,0,0,0,1,0,0,0,0,0,0,0,1,0
5,35.000000,0,0,8.0500,0,0,1,0,0,0,0,0,0,0,1,0,1,0
6,29.881138,0,0,8.4583,0,0,1,0,0,0,0,0,0,0,1,1,0,0
7,54.000000,0,0,51.8625,0,0,0,0,0,0,1,0,0,0,0,0,1,0
8,2.000000,3,1,21.0750,0,0,1,0,0,0,0,0,0,0,1,0,1,0
9,27.000000,0,2,11.1333,1,0,1,0,0,0,0,0,0,0,1,0,1,0


In [54]:
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X, y).coef_

array([[-3.16484758e-02, -2.78781236e-01, -9.20307108e-02,
         2.90040618e-03,  3.62068094e+00, -2.90667002e-01,
        -1.15647961e+00,  7.48109689e-02, -2.88008923e-01,
         2.95219201e-01,  7.26533192e-01,  1.73031274e-01,
        -8.11892835e-01, -1.95027531e-01, -7.42552797e-01,
         1.31675151e-01, -2.38548344e-01,  1.09840866e-01]])

In [47]:
#logreg = LogisticRegression(max_iter=200)
logreg = LogisticRegression(solver='liblinear')
rfe = RFE(logreg, 18, step= 1)
rfe = rfe.fit(X, y)

In [57]:
rfe.support_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [58]:
rfe.ranking_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [59]:
rfe.ranking_
columnas_RFE = np.array(X.columns)[rfe.support_]
X = X[columnas_RFE]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [60]:
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [61]:
y_pred = logreg.predict(X_test)
logreg.score(X_test, y_test)

0.851145038167939

In [62]:
matriz_confusion = confusion_matrix(y_test, y_pred)
matriz_confusion

array([[142,  17],
       [ 22,  81]])

In [63]:
data_cleaned.describe()

,Age,SibSp,Parch,Fare,Survived,isfemale
count,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,29.881138,0.498854,0.385027,33.295479,0.377387,0.355997
std,12.883193,1.041658,0.865560,51.738879,0.484918,0.478997
min,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,0.000000,0.000000,7.895800,0.000000,0.000000
50%,29.881138,0.000000,0.000000,14.454200,0.000000,0.000000
75%,35.000000,1.000000,0.000000,31.275000,1.000000,1.000000
max,80.000000,8.000000,9.000000,512.329200,1.000000,1.000000


In [64]:
set(data_cleaned.Pclass)

{'First', 'Second', 'Third'}

In [65]:
set(data_cleaned.Cabin_class)

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'U'}

In [66]:
set(data_cleaned.Embarked)


{'C', 'Q', 'S', 'U'}